In [3]:
'''
influence-analysis.ipynb
Analysing the relationship between places of issuing the documents and the places mentioned as benefitiaries and donations
'''

import os
import pandas as pd
import geopandas as gpd
import pathlib
from shapely import wkt
from shapely.geometry import Point, Polygon, LineString, MultiPoint

import geojson
import h3

from geovoronoi import voronoi_regions_from_coords
crs4326 = {'init': 'epsg:4326'}

In [4]:
DIR_ROOT = os.path.join(pathlib.Path().absolute(), '../../..' )

influences_df = pd.read_csv(DIR_ROOT + '/data/04_itinerary/influences.csv')
influences_df = influences_df[influences_df['domestic'] == True]

localities = pd.read_csv(DIR_ROOT + '/data/02_processed/localities.csv')
localities['geometry'] = localities['geometry'].apply(wkt.loads)
localities = gpd.GeoDataFrame(localities, crs=crs4326).set_geometry('geometry')

/home/adam/miniconda3/envs/po2/lib/python3.9/site-packages/pyproj/crs/crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [5]:
bbox = localities.buffer(1).total_bounds
bounds = dict(geojson.Polygon([[[bbox[0], bbox[1]], [bbox[0], bbox[3]], [bbox[2], bbox[3]], [bbox[2], bbox[1]]]]))
hex_ids = list(h3.polyfill(bounds, 4))

hex_df = gpd.GeoDataFrame(
    [{"geometry": Polygon(h3.h3_to_geo_boundary(x)), "id": x} for x in hex_ids], crs=crs4326
)
hex_df.set_index('id', inplace=True)

hex_df['id'] = hex_df.index
hex_df['geometry'] = hex_df.apply(lambda x: Polygon(h3.h3_to_geo_boundary(x['id'])), axis=1)
hex_df

/tmp/ipykernel_99614/1308781547.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  bbox = localities.buffer(1).total_bounds


,geometry,id
id,,
8452615ffffffff,"POLYGON ((20.37162 51.60373, 20.17768 51.47443...",8452615ffffffff
8463315ffffffff,"POLYGON ((17.39234 55.61343, 17.20596 55.68906...",8463315ffffffff
8452a97ffffffff,"POLYGON ((14.35470 42.01205, 14.14732 41.89267...",8452a97ffffffff
8462b19ffffffff,"POLYGON ((6.50370 54.11325, 6.69110 54.16840, ...",8462b19ffffffff
84520b1ffffffff,"POLYGON ((16.96423 46.27770, 16.76134 46.15324...",84520b1ffffffff
...,...,...
8452cd3ffffffff,"POLYGON ((11.90443 48.95005, 11.71239 48.83093...",8452cd3ffffffff
847a903ffffffff,"POLYGON ((7.46363 50.51598, 7.65383 50.57729, ...",847a903ffffffff
8443941ffffffff,"POLYGON ((20.59093 54.49746, 20.39527 54.57958...",8443941ffffffff


In [6]:
def get_influence_origins(hex_geo):
    localities_in_hex = localities[localities.intersects(hex_geo)]

    infl_list = []
    for lihi, locality_in_hex in localities_in_hex.iterrows():
        inf_object_rows = influences_df[influences_df['name'] == locality_in_hex['name']]
        infl_list.extend(inf_object_rows['locality'].tolist())

    return { x: infl_list.count(x) for x in infl_list}

hex_df['infl'] = hex_df.apply(
    lambda x: get_influence_origins(x['geometry']), 
    axis=1
)
print(hex_df['infl'].tolist())

hex_df['all'] = hex_df.apply(
    lambda x: sum(x['infl'].values()), 
    axis=1
)

centers_to_check = ['Praha', 'Brno', 'Wien', 'Graz', 'Opava', 'Linz', 'Kostanjevica na Krki', 'Friesach']


for center in centers_to_check:
    hex_df[center] = hex_df.apply(
        lambda x: x['infl'][center] if center in x['infl'] else 0, 
        axis=1
    )

hex_df['others'] = hex_df.apply(
    lambda x: x['all'] - sum(x[center] for center in centers_to_check) , 
    axis=1
)



[{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {'Praha': 2, 'Brno': 2, 'Jihlava': 1}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {'Praha': 1}, {}, {'Brno': 1}, {}, {'Krems an der Donau': 1, 'Linz': 6, 'Graz': 1, 'Wien': 2, 'Znojmo': 1, 'Freistadt': 1, 'Litoměřice': 1}, {}, {}, {}, {}, {}, {}, {}, {'Brno': 1, 'Olomouc': 2, 'Přištpo': 1}, {}, {}, {}, {}, {'Zbraslav': 1}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {'Znojmo': 1, 'Praha': 2}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {'Písek': 1}, {}, {'Praha': 2, 'Písek': 2, 'Brno': 2}, {}, {}, {'Ljubljana': 1}, {}, {'Kamýk nad Vltavou': 1, 'Stična': 1, 'Praha': 1, 'Brno': 2, 'Znojmo': 1, 'Wien': 3}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {'Wien': 2, 'Krems an der Donau': 1, 'Linz': 1}, {'Praha': 1}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, 

In [7]:

hex_df.reset_index(drop=True).to_file(DIR_ROOT + '/data/06_outputs/hex_influence.geojson', driver="GeoJSON")

hex_df_geo = hex_df.copy()
hex_df_geo['centroid'] = hex_df_geo.apply(
    lambda x: x['geometry'].centroid, 
    axis=1
)
hex_df_geo = hex_df_geo.drop(columns=["geometry"])
hex_df_geo.set_geometry('centroid', inplace=True)
hex_df_geo = hex_df_geo.reset_index(drop=True)
print(hex_df_geo)
hex_df_geo.to_file(DIR_ROOT + '/data/06_outputs/hex-centroids_influence.geojson', driver="GeoJSON")


                   id infl  all  Praha  Brno  Wien  Graz  Opava  Linz  \
0     8452615ffffffff   {}    0      0     0     0     0      0     0   
1     8463315ffffffff   {}    0      0     0     0     0      0     0   
2     8452a97ffffffff   {}    0      0     0     0     0      0     0   
3     8462b19ffffffff   {}    0      0     0     0     0      0     0   
4     84520b1ffffffff   {}    0      0     0     0     0      0     0   
...               ...  ...  ...    ...   ...   ...   ...    ...   ...   
1973  8452cd3ffffffff   {}    0      0     0     0     0      0     0   
1974  847a903ffffffff   {}    0      0     0     0     0      0     0   
1975  8443941ffffffff   {}    0      0     0     0     0      0     0   
1976  845246bffffffff   {}    0      0     0     0     0      0     0   
1977  84520b9ffffffff   {}    0      0     0     0     0      0     0   

      Kostanjevica na Krki  Friesach  others                   centroid  
0                        0         0       0  POI